In [1]:
import os
os.chdir('/home/vaschetti/maxarSrc/creating_labels/models/EfficientSAM')
from efficient_sam.build_efficient_sam import build_efficient_sam_vitt, build_efficient_sam_vits
from PIL import Image
from torchvision import transforms
import torch
import numpy as np
import zipfile

In [2]:
device = 'cuda:3'
models = {}
# Build the EfficientSAM-Ti model.
models['efficientsam_ti'] = build_efficient_sam_vitt().to(device)

# Since EfficientSAM-S checkpoint file is >100MB, we store the zip file.
with zipfile.ZipFile("weights/efficient_sam_vits.pt.zip", 'r') as zip_ref:
    zip_ref.extractall("weights")
# # Build the EfficientSAM-S model.
models['efficientsam_s'] = build_efficient_sam_vits().to(device)

In [3]:
# load an image
sample_image_np = np.array(Image.open("figs/examples/dogs.jpg"))
sample_image_tensor = transforms.ToTensor()(sample_image_np)
sample_image_tensor = sample_image_tensor.to(device)

In [4]:
next(models['efficientsam_ti'].parameters()).is_cuda

True

In [5]:
next(models['efficientsam_s'].parameters()).is_cuda

True

In [16]:
from time import time
# Feed a few (x,y) points in the mask as input.

input_points = torch.tensor([[[[580, 350], [650, 350]]]], device=device)
print(input_points.device)

input_labels = torch.tensor([[[1, 1]]], device=device)
print(input_labels.device)

print('sample_img tensor device: ', sample_image_tensor.device)

# Run inference for both EfficientSAM-Ti and EfficientSAM-S models.
for model_name, efficient_sam in models.items():
    print('Running inference using ', model_name)
    t_0 = time()
    predicted_logits, predicted_iou = efficient_sam(
        sample_image_tensor[None, ...],
        input_points,
        input_labels,
    )
    print('Time taken: ', time() - t_0)
    # The masks are already sorted by their predicted IOUs.
    # The first dimension is the batch size (we have a single image. so it is 1).
    # The second dimension is the number of masks we want to generate (in this case, it is only 1)
    # The third dimension is the number of candidate masks output by the model.
    # For this demo we use the first mask.
    mask = torch.ge(predicted_logits[0, 0, 0, :, :], 0).cpu().detach().numpy()
    masked_image_np = sample_image_np.copy().astype(np.uint8) * mask[:,:,None]
    #Image.fromarray(masked_image_np).save(f"figs/examples/dogs_{model_name}_mask.png")
    


cuda:3
cuda:3
sample_img tensor device:  cuda:3
Running inference using  efficientsam_ti
Time taken:  0.26722145080566406
Running inference using  efficientsam_s
Time taken:  0.2212996482849121
